## load libs

In [64]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from process_images import *
from pystripe.core import *
import matplotlib.pyplot as plt
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    if len(img_list) == 1:
        plt.figure(figsize=(20, 20))
        plt.imshow(img_list[0], cmap='gray', vmin=0, vmax=vmax)
        plt.title(img_labels[0])
    else:
        fig, axes = plt.subplots(nrows=1, ncols=len(img_list), figsize=(20, 20))
        for idx, (im, label) in enumerate(zip(img_list, img_labels)):
            axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
            axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

In [ ]:
from parallel_image_processor import *
tsv_volume = TSVVolume.load(r'E:\20230510_13_34_13_SM230308_05_LS_15x_800z_MIP_stitched\Ex_488_Em_525_MIP_xml_import_step_5.xml')
shape: Tuple[int, int, int] = tsv_volume.volume.shape  # shape is in z y x format
img = tsv_volume.imread(
    VExtent(
        tsv_volume.volume.x0, tsv_volume.volume.x1,
        tsv_volume.volume.y0, tsv_volume.volume.y1,
        tsv_volume.volume.z0 + shape[0]//2, tsv_volume.volume.z0 + shape[0]//2 + 1),
    tsv_volume.dtype)[0]
parallel_image_processor(
    source=TSVVolume.load(r'/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_xml_import_step_5.xml'),
    destination=r"/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_tif",
    fun=process_img,
    kwargs={'bleach_correction_frequency': 0.0005, 'bleach_correction_max_method': False, 'bleach_correction_y_slice_max': None, 'threshold': None, 'sigma': (4000.0, 4000.0), 'bidirectional': True, 'lightsheet': False, 'percentile': 0.25, 'rotate': 90, 'convert_to_8bit': False, 'bit_shift_to_right': 8, 'tile_size': (39220, 28056), 'd_type': 'uint16', "verbose": True},
    source_voxel=(0.8, 0.4, 0.4),
    target_voxel=20,
    max_processors=1
)

In [ ]:
def get_layer(
    index: int,          # layer of image requested
    image: ndarray,      # 3-D image (use TifStack.as_3d_numpy())
    plane = "xy",        # must be "xy", "yx", "xz", "zx", "yz", "zy"
    img_format = "zyx",  # xyz in some order
):
    # guards
    if plane not in {"xy", "yx", "xz", "zx", "yz", "zy"} or img_format not in {"zyx", "zxy", "yxz", "yzx", "xyz", "xzy"}:
        print(f"Invalid plane selected in get_layer().  Plane: {plane}, Layer: {index}, Img_format: {img_format}\nReturning to caller...")
        return None

    # get the layer
    if 'x' not in plane:   sub = img_format.index('x')
    elif 'y' not in plane: sub = img_format.index('y')
    elif 'z' not in plane: sub = img_format.index('z')

    if sub == 0:   layer_image = image[index, :, :]
    elif sub == 1: layer_image = image[:, index, :]
    elif sub == 2: layer_image = image[:, :, index]

    # if plane is flipped compared to image format, return the transpose.
    if plane not in (img_format[:sub] + img_format[sub + 1:]):
        return layer_image.transpose()
    return layer_image

# run to rotate images
from os import makedirs

######## CHANGE THESE ########
cha1_path = "D:/BMAP/Brain 4/cha1"
cha2_path = "D:/BMAP/Brain 4/cha2"
cha3_path = "D:/BMAP/Brain 4/cha3"
# set to None if nothing to convert

##############################
if cha1_path:
    makedirs(cha1_path + "_zx", exist_ok=True)
    makedirs(cha1_path + "_zy", exist_ok=True)
    stack1 = TifStack(cha1_path).as_3d_numpy()
    for i in range(stack1.shape[1]):
        imwrite(cha1_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack1, "zx"))
    for i in range(stack1.shape[2]):
        imwrite(cha1_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack1, "zy"))
if cha2_path:
    makedirs(cha2_path + "_zx", exist_ok=True)
    makedirs(cha2_path + "_zy", exist_ok=True)
    stack2 = TifStack(cha2_path).as_3d_numpy()
    for i in range(stack2.shape[1]):
        imwrite(cha2_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack2, "zx"))
    for i in range(stack2.shape[2]):
        imwrite(cha2_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack2, "zy"))
if cha3_path:
    makedirs(cha3_path + "_zx", exist_ok=True)
    makedirs(cha3_path + "_zy", exist_ok=True)
    stack3 = TifStack(cha3_path).as_3d_numpy()
    for i in range(stack3.shape[1]):
        imwrite(cha3_path + "_zx/" + str(i + 1) + ".tif", get_layer(i, stack3, "zx"))
    for i in range(stack3.shape[2]):
        imwrite(cha3_path + "_zy/" + str(i + 1) + ".tif", get_layer(i, stack3, "zy"))
print("Operation Completed")

In [18]:
from pathlib import Path
from align_images import get_layer
from numpy import min, max, uint8, zeros_like, ndarray, multiply

def write_to_file(images: list[ndarray], filepath: Path):
    filepath.mkdir(parents=True, exist_ok=True)
    for n, image in enumerate(images):
        local = filepath / f'cha{n}'
        local.mkdir(parents=True, exist_ok=True)
        for layer in range(image.shape[0]):
            path = local.absolute() / (str(layer) + ".tif")
            imwrite(path, get_layer(layer, image, "yx"))
        print("wrote to file")
        
# written by ChatGPT
def normalize_array_inplace(arr: ndarray):
    min_val = min(arr)
    max_val = max(arr)

    arr -= min_val
    arr /= (max_val - min_val)

    # Scale the values to be between 0 and 255
    arr *= 255
    arr.astype(uint8, copy=False)
    

# multiplies two 2d-ndarrays, saving solution in arr1 instead of allocating more memory
def mult_in_place(arr1, arr2):
    for r in range(arr1.shape[0]):
        for c in range(arr1.shape[1]):
            arr1[r][c] *= arr2[r][c]
 
 
def get_borders(img: ndarray, copy=False):
    print("get_borders")
    mask = zeros_like(img)
    for ind in range(img.shape[0]):
        print(f'layer {ind}')
        mask[ind] = get_img_mask(img[ind], otsu_threshold(img[ind]))
    multiply(img, mask, out=img)
    
    

In [ ]:
from align_images import align_all_images, resize_arrays, get_borders
from supplements.tifstack import TifStack

cha1 = TifStack("C:/Users/ajan/Pictures/cha0").as_3d_numpy()
cha2 = TifStack("C:/Users/ajan/Pictures/cha1").as_3d_numpy()
cha3 = TifStack("C:/Users/ajan/Pictures/cha2").as_3d_numpy()
output_path = Path("D:/aligned_images/align_test")
max_iterations = 50

print("Images loaded")

# make arrays the same size
channels = resize_arrays([cha1, cha2, cha3])

print("Images resized")
# print("test")
# for i in channels: print(i.shape)


for img in channels:
    get_borders(img)    

print("got borders")

# align images
alignments, residuals = align_all_images(channels, verbose=True, make_copy=False)

# normalize images and convert to uint8
for channel in channels: normalize_array_inplace(channel)

print("Images normalized")

write_to_file(channels, output_path)

print("Done!")
# print("Alignment: ", end='')
# print(alignments)
# print("Residuals: ", end='')
# print(residuals) 

* try using mask for better edge detection
* Sobel operator -> mask -> align

In [2]:
# convert stacks to .ims
import os
dir_name = r"D:\aligned_images\cube_test\big_cube\downsampled"

os.system(f'python convert.py -i "{dir_name}/cha0" -o "{dir_name}/cha0.ims" -dx 2 -dy 2 -dz 2')
os.system(f'python convert.py -i "{dir_name}/cha1" -o "{dir_name}/cha1.ims" -dx 2 -dy 2 -dz 2')
os.system(f'python convert.py -i "{dir_name}/cha2" -o "{dir_name}/cha2.ims" -dx 2 -dy 2 -dz 2')

0

In [ ]:
!python align_images.py C:/Users/ajan/Pictures/ D:\aligned_images\align_test
!echo done

In [22]:
from align_images import resize_arrays

from os import system
from matplotlib.pyplot import imshow, show

pad_only = False
generate_ims = False
dx, dy, dz = 0, 0, 0
filepaths = ["C:/Users/ajan/Pictures/cha0", "C:/Users/ajan/Pictures/cha1","C:/Users/ajan/Pictures/cha2"]
output_file = f"C:/Users/ajan/Pictures/align_test"
# Image Processing --------------------------------------------------------------------
print("Loading images...")
count = 0
try:
    cha1 = TifStack(filepaths[0]).as_3d_numpy()
    count = 1
    cha2 = TifStack(filepaths[1]).as_3d_numpy()
    count = 2
    cha3 = TifStack(filepaths[2]).as_3d_numpy()
    print("Images loaded")
except Exception:
    print(f"Error: Invalid TifStack found at {filepaths[count]}")
    exit(1)

output_path = Path(output_file)
output_path.mkdir(parents=True, exist_ok=True)

print(filepaths)

imshow(cha1[0])
show(block=True)
imshow(cha2[0])
show(block=True)
imshow(cha3[0])
show(block=True)

print("Resizing images...")

channels = resize_arrays([cha1, cha2, cha3])
print("Images resized")

# normalize images and convert to uint8
# print("Normalizing images")
# for channel in channels: normalize_array_inplace(channel)



if not pad_only:
    print("Aligning images... (this may take a while)")
    # TODO: FIX THIS
    for i in range(len(channels)):
        # channels[i] *= (channels[i] > percentile(channels[i], 80))  # set all pixels below threshold to zero. (weeds out noise along edges)
        # get_borders(channels[i])
        # channels[i] = sobel(channels[i])

        print(channels[i])
        imshow(channels[i][0, :, :])
        show()


    # # align images
    # alignments, residuals = align_all_images(channels, max_iter=max_iterations, verbose=False, make_copy=False)
    # print("Images aligned")


print("Images normalized")

print("Writing to file")
write_to_file(channels, output_path)
print("Wrote to file")

if generate_ims:
    print("Generating .ims files")
    system(f'python convert.py -i "{output_path}/cha0" -o "{output_path}/cha0.ims" -dx {dx} -dy {dy} -dz {dz}')
    system(f'python convert.py -i "{output_path}/cha1" -o "{output_path}/cha1.ims" -dx {dx} -dy {dy} -dz {dz}')
    system(f'python convert.py -i "{output_path}/cha2" -o "{output_path}/cha2.ims" -dx {dx} -dy {dy} -dz {dz}')
    print(".ims files created")

print("Alignments:")
# print(alignments)
print("\n\nOperation completed.")

AttributeError: partially initialized module 'pandas' has no attribute 'plotting' (most likely due to a circular import)

In [2]:
from numpy import zeros, full, pad, uint8 
from align_images import write_to_file, roll_pad
from pathlib import Path
from tifffile import imwrite

# do not run this code unless you're sure you want to generate like 1TB of data
CUBE_SL = 4000
IMG_SL = 8000 
WALL_THICKNESS = 200  # set to 0 for completely filled cube
filepath = Path(r'D:\aligned_images\cube_test\big_cube\cha0')

top_padding, bottom_padding = ((IMG_SL - CUBE_SL) // 2, (IMG_SL - CUBE_SL + 1) // 2)
layer_num = 0
filepath.mkdir(parents=True, exist_ok=True)

# top padding
top_pad_image = zeros((IMG_SL, IMG_SL), dtype=uint8)
for i in range(top_padding):
    imwrite(filepath / f"{layer_num}.tif", top_pad_image, dtype=uint8)
    layer_num += 1

# cube
if WALL_THICKNESS:
    top_edge_image = full((CUBE_SL, CUBE_SL), 255, dtype=uint8)
    top_edge_image = pad(top_edge_image, ((top_padding, bottom_padding), (top_padding, bottom_padding)), mode='constant', constant_values=0)
    for i in range(WALL_THICKNESS):
        imwrite(filepath / f"{layer_num}.tif", top_edge_image, dtype=uint8)
        layer_num += 1
    cube_image = zeros((CUBE_SL - 2 * WALL_THICKNESS, CUBE_SL - 2 * WALL_THICKNESS), dtype=uint8)
    cube_image = pad(cube_image, WALL_THICKNESS, mode='constant', constant_values=255)
    cube_image = pad(cube_image, ((top_padding, bottom_padding), (top_padding, bottom_padding)), mode='constant', constant_values=0)
    for i in range(CUBE_SL - 2 * WALL_THICKNESS):
        imwrite(filepath / f"{layer_num}.tif", cube_image, dtype=uint8)
        layer_num += 1
    bottom_edge_image = full((CUBE_SL, CUBE_SL), 255, dtype=uint8)
    bottom_edge_image = pad(bottom_edge_image, ((top_padding, bottom_padding), (top_padding, bottom_padding)), mode='constant', constant_values=0)
    for i in range(WALL_THICKNESS):
        imwrite(filepath / f"{layer_num}.tif", top_edge_image, dtype=uint8)
        layer_num += 1
else:
    filled_cube_image = full((CUBE_SL, CUBE_SL), 255, dtype=uint8)
    filled_cube_image = pad(filled_cube_image, ((top_padding, bottom_padding), (top_padding, bottom_padding)), mode='constant', constant_values=0)
    for i in range(CUBE_SL):
        imwrite(filepath / f"{layer_num}.tif", filled_cube_image, dtype=uint8)
        layer_num += 1

# bottom padding
bottom_pad_image = zeros((IMG_SL, IMG_SL), dtype=uint8)
for i in range(bottom_padding):
    imwrite(filepath / f"{layer_num}.tif", bottom_pad_image, dtype=uint8)
    layer_num += 1


# cube = zeros((126, 126, 126), dtype=uint8)
# cube = pad(cube, 1, mode='constant', constant_values=255)
# 
# cha0 = pad(cube, 64)
# cha1 = pad(cube, 64)
# cha2 = pad(cube, 64)
# 
# roll_pad(cha1, 1, axis=0)
# roll_pad(cha1, 2, axis=1)
# roll_pad(cha1, 3, axis=2)
# 
# roll_pad(cha2, -3, axis=0)
# roll_pad(cha2, -2, axis=1)
# roll_pad(cha2, -1, axis=2)
# 
# write_to_file([cha0, cha1, cha2], path, data_type='uint8', verbose=True)
    


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from process_images import *
from pystripe.core import *
import matplotlib.pyplot as plt
from math import exp
from numpy import vectorize, amax, amin
from copy import deepcopy
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    if len(img_list) == 1:
        plt.figure(figsize=(20, 20))
        plt.imshow(img_list[0], cmap='gray', vmin=0, vmax=vmax)
        plt.title(img_labels[0])
    else:
        fig, axes = plt.subplots(nrows=1, ncols=len(img_list), figsize=(20, 20))
        for idx, (im, label) in enumerate(zip(img_list, img_labels)):
            axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
            axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

def sigmoid(x):
  return 255 / (1 + exp(-10* x))

v_sigmoid = vectorize(sigmoid)
img = imread_tif_raw_png(Path(r"D:\aligned_images\20230825_SM230601_06_LS_15x_800z_B6\orig\cha0\111.tif"))
sobel_img = sobel(img)
print("Max: ", amax(sobel_img))
print("Min: ", amin(sobel_img))

threshold = threshold_multiotsu(sobel_img, classes=4)[2]
s_img = v_sigmoid(sobel_img)

mask = get_img_mask(s_img, threshold, close_steps=50, open_steps=5, flood_fill_flag=4)
plot_images([img, sobel_img, mask], ["orig", "sobel", "mask"], threshold)  # mask*threshold

In [21]:
from align_images import roll_pad
from pathlib import Path  
from tifffile import natural_sorted, imread, imwrite
from numpy import zeros

def shift_big_image(file_path_input: Path, file_path_output: Path, offset: int, axis): # (axis 0 = z, axis 1 = y, axis 2 = x)
    if offset == 0: return
    file_path_output.mkdir(parents=True, exist_ok=True)
    files = natural_sorted([file.__str__() for file in file_path_input.iterdir() if
                      file.is_file() and file.suffix.lower() in (".tif", ".tiff")])
    first_file = imread(files[0])
    
    file_shape = first_file.shape
    data_type = first_file.dtype
    num_files = len(files)
    
    layer_num = 0
    
    print(file_shape)
    
    if axis == 0: # z-axis
        if offset > 0:
            # front padding
            while layer_num < offset:
                output_file = file_path_output / f"{layer_num}.tif"
                imwrite(output_file, zeros(file_shape, dtype=data_type), dtype=data_type)
                layer_num += 1
            # rest of the files
            input_layer = 0
            while layer_num < num_files:
                output_file = file_path_output / f"{layer_num}.tif"
                temp_file = imread(files[input_layer])
                imwrite(output_file, temp_file, dtype=data_type)
                layer_num += 1
                input_layer += 1
        else:
            # files
            input_layer = -offset
            while layer_num < num_files + offset:
                output_file = file_path_output / f"{layer_num}.tif"
                temp_file = imread(files[input_layer])
                imwrite(output_file, temp_file, dtype=data_type)
                layer_num += 1
                input_layer += 1
            # padding
            while layer_num < num_files:
                output_file = file_path_output / f"{layer_num}.tif"
                imwrite(output_file, zeros(file_shape, dtype=data_type), dtype=data_type)
                layer_num += 1
    else:
        for layer_num, file in enumerate(files):
            output_file = file_path_output / f"{layer_num}.tif"
            temp_file = imread(file)
            roll_pad(temp_file, offset, axis - 1)
            imwrite(output_file, temp_file, dtype=data_type)
            
            
shift_big_image(Path(r'D:\aligned_images\cube_test\big_cube\cha0'), Path(r'D:\aligned_images\cube_test\big_cube\cha1'), -50, 2)
            

(256, 256)


In [2]:
from align_images import process_big_image
from pathlib import Path

process_big_image(Path(r'D:\aligned_images\cube_test\big_cube\cha0'), Path(r'D:\aligned_images\cube_test\big_cube\cha2'), Path(r'D:\aligned_images\cube_test\big_cube\cha0'), (8000, 8000, 8000), [-30, -20, -10])

In [4]:
!convert.py --input D:\aligned_images\cube_test\big_cube\cha0 --tif D:\aligned_images\cube_test\big_cube\downsampled\cha0 -dx 2 -dy 2 -dz 2 -dt 20

Traceback (most recent call last):
  File "C:\Users\ajan\Documents\GitHub\image-preprocessing-pipeline\convert.py", line 11, in <module>
    import psutil
ModuleNotFoundError: No module named 'psutil'


In [66]:
# converts original to downsampled
# !python convert.py --input D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\Ex_488_Em_525_tif --tif D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_488_Em_525_tif --downsample_path D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_488_Em_525_tif -dsx 10 -dsy 10 -dsdt uint8 -dx 2 -dy 2 -dz 2 -dt 20
# !python convert.py --input D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\Ex_561_Em_600_tif --tif D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_561_Em_600_tif --downsample_path D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_561_Em_600_tif -dsx 10 -dsy 10 -dsdt uint8 -dx 2 -dy 2 -dz 2 -dt 20
!python convert.py --input D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\Ex_642_Em_680_tif --tif D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_642_Em_680_tif --downsample_path D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_642_Em_680_tif -dsx 10 -dsy 10 -dsdt uint8 -dx 2 -dy 2 -dz 2 -dt 20

Modifying destination: D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_642_Em_680_tif
2024-06-15 13:37:11: down-sampling: 
	post-processed shape zyx: 900.0 1686.0 1266.0
	actual voxel sizes zyx: 20.0 20.0 20.0
Modifying downsampled_path: D:\aligned_images\20220622_SW220414_02_LS_6x_1000z\downsampled\Ex_642_Em_680_tif\Ex_642_Em_680_tif_z20.0_yx20.0um
2024-06-15 13:37:11: starting workers ...
2024-06-15 13:37:12: using 20 workers. 9000 images need to be processed.
image processing function changed the voxel size from (2.0, 2.0, 2.0) to (2.0, 19.989330171902786, 19.990528808208367)
image processing function changed the voxel size from (2.0, 2.0, 2.0) to (2.0, 19.989330171902786, 19.990528808208367)
image processing function changed the voxel size from (2.0, 2.0, 2.0) to (2.0, 19.989330171902786, 19.990528808208367)
image processing function changed the voxel size from (2.0, 2.0, 2.0) to (2.0, 19.989330171902786, 19.990528808208367)
image processing function changed the v


 workers: 100%|##########| 20/20 [00:00<00:00, 180.18it/s]

 ImgProc: 100%|##########| 9000/9000 [18:54<00:00,  7.94 images/s]

loading: 100%|##########| 900/900 [00:04<00:00, 202.27images/s]


In [62]:
from align_images import pad_to_shape, trim_to_shape, roll_pad
from numpy import stack, zeros
from pathlib import Path
from tifffile import natural_sorted, imread, imwrite
# offsets must have shape (D, 3), where D is the number of file path inputs
# offsets[reference_index] = [0, 0, 0]
def process_big_images(file_path_inputs: list[Path], file_path_output: Path, reference_index: int, offsets: list[list[int]], save_singles=False):
    # load image paths
    file_paths = []
    for input_path in file_path_inputs:
        temp = natural_sorted([file.__str__() for file in input_path.iterdir() if file.is_file() and file.suffix.lower() in (".tif", ".tiff")])
        print(list(input_path.iterdir()))
        file_paths.append(temp)
    
    image_shapes = [] # list[tuple]
    data_type = imread(file_paths[reference_index][0]).dtype
    for image in file_paths:
        temp_shape = (len(image), *imread(image[0]).shape)
        image_shapes.append(temp_shape)
        
    # calculate operation shape
    operation_shape = [max(dim) for dim in zip(*image_shapes)]
    
    # calculate pad amounts
    pad_to_max = []
    for image in range(len(file_paths)):
        pad_dim = [operation_shape[i] - image_shapes[image][i] for i in range(len(image_shapes[image]))]
        pad_to_max.append(list(map(lambda x: (x // 2, (x + 1) // 2), pad_dim)))
    
    # process layers
    for n_ref in range(len(file_paths[reference_index])):
        # calculate which orig layer to use for given final layer
        n_orig = []
        for i in range(len(file_paths)):
            if i == reference_index: 
                n_orig.append(n_ref)
                continue
            n_orig.append(n_ref + pad_to_max[reference_index][0][0] - pad_to_max[i][0][0] - offsets[i][0]) 
        
        # print(n_orig)
        combined_image = []
        for count, n_img in enumerate(n_orig):
            if 0 <= n_img < image_shapes[count][0]:
                file = imread(file_paths[count][n_img])
                # pad image to operation dimensions
                pad_to_shape(operation_shape[1:], file)
                
                # shift image in x and y directions
                roll_pad(file, offsets[count][1], axis=0)
                roll_pad(file, offsets[count][2], axis=1)
                
                # reshape image to reference dimensions
                trim_to_shape(image_shapes[reference_index][1:], file)
                
                combined_image.append(file)
            else: 
                # save zeroes for that layer if out of bounds
                combined_image.append(zeros(image_shapes[reference_index][1:]))
                
        # save RGB image
        local = file_path_output / 'RGB'
        local.mkdir(parents=True, exist_ok=True)
        composite = stack(combined_image, axis=-1)
        
        path = local.absolute() / Path(file_paths[reference_index][n_ref]).name
        imwrite(path, composite.astype(data_type), dtype=data_type)
    
        # save individual images
        if save_singles:
            for n, channel_img in enumerate(combined_image):
                # print(n)
                local = file_path_output / file_path_inputs[n].name
                local.mkdir(parents=True, exist_ok=True)
                path = local.absolute() / Path(file_paths[reference_index][n_ref]).name
                # print(path)
                imwrite(path, channel_img.astype(data_type), dtype=data_type)
    
    print("completed")

In [63]:
inputs = [Path(r"D:\aligned_images\cube_test\big_cube\downsampled\cha0"),
          Path(r"D:\aligned_images\cube_test\big_cube\downsampled\cha1"),
          Path(r"D:\aligned_images\cube_test\big_cube\downsampled\cha2")]

output = Path(r"D:\aligned_images\cube_test\big_cube\downsampled\big_cube_test_out")
ref = 0
offsets = [[0, 0, 0], [-1, -2, -3], [3, 2, 1]]
save_singles = True

process_big_images(inputs, output, ref, offsets, save_singles)



[WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000000.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000001.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000002.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000003.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000004.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000005.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000006.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000007.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000008.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000009.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/cha0/img_000010.tif'), WindowsPath('D:/aligned_images/cube_test/big_cube/downsampled/ch